<a href="https://colab.research.google.com/github/MatiasBlanco0/Trabajo-Practico-Netflix/blob/main/Trabajo_Practico_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importante

## Librerias

In [43]:
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

Importamos `files` de `google.colab` para poder subir el archivo csv.

Importamos `pandas` para poder leer y modificar el archivo csv.

Importamos `matplotlib.pyplot` para poder hacer graficos

## Subir Archivo CSV

In [2]:
uploaded = files.upload()

Saving NetflixHistory.csv to NetflixHistory.csv


## Leer y preparar el archivo CSV

Guardamos el archivo csv en una variable llamada `df`por dataframe

In [29]:
df = pd.read_csv("/content/NetflixHistory.csv")

Correr esto para filtrar los dispositivos indicados. `inplace=True` sirve para que la funcion sobreescriba el data frame en vez de devolver una copia

In [30]:
devicesToIgnore = ['Sony PS4', 'Netflix Windows App - Cadmium Windows Mobile', 'Apple iPhone 6s', 'Apple iPhone 7 Plus (CDMA)', 'Chrome PC (Cadmium)', 'Samsung 2015 NT14M DTV Smart TV', 'iPhone 5S GSM', 'Google Chromecast V3 Streaming Stick']
for deviceToIgnore in devicesToIgnore:
  df.drop(df[df['Device Type'] == deviceToIgnore].index, inplace=True)

Correr una vez, sino tira error. Porfa 🙏

In [31]:
def convertTextToSeconds(string):
  # El string de duration es como '01:20:14'
  hours, minutes, seconds = string.split(":")
  return int(seconds) + (int(minutes) + int(hours) * 60) * 60

# Reemplazo duration con la duracion en segundos
df['Duration'] = df['Duration'].apply(convertTextToSeconds)

Separar peliculas y series en distintos data frames

In [67]:
# Filtrado los trailers, hooks, recaps y los teaser_trailers
df = df[pd.isna(df['Supplemental Video Type'])]
series = df[df['Title'].str.contains("Capítulo|Season")]
peliculas = df[~df['Title'].str.contains("Capítulo|Season")]

# Ejercicios

## A. Contabilizar el dispositivo con el que más títulos se reprodujeron.

Nosotros contamos como titulo al titulo unico, es decir, si se ve una pelicula dos veces, esto cuenta como una vez

In [41]:
devices = {}
# Iteramos por cada dispositivo del dataframe y si no esta lo agregamos con un valor
for device in df['Device Type']:
  if device not in devices:
    # Sacamos la cuenta de valores de la serie Title donde el Device Type es igual al del device actual, lo convertimos a un diccionario y sacamos las keys,
    # porque es lo que nos interesa (los nombres de los titulos)
    # Al final devolvemos el largo para saber cuantos titulos se vieron en ese dispositivo
    devices[device] = len(df[df['Device Type'] == device]['Title'].value_counts().to_dict().keys())

# Utilizamos la funcion sorted para ordenar en base a los valores, accediendolos mediante devices.items() y en el callback de key devolviendo el valor.
# Ademas de decir que los ordene de mayor a menor
# Esto devuelve una lista de tuplas, hacemos un slice para solo recibir el primer elemento, por eso despues los convertimos a un diccionario
devices = dict(sorted(devices.items(), key=lambda item: item[1], reverse=True)[:1])
devices

{'Apple Apple TV 4 Apple TV': 1920}

El dispositivo en el que mas titulos se reprodujeron fue: Apple Apple TV 4 Apple TV, con 1920 titulos

## B. Encontrar la película con más ocurrencias y la película con mayor duración total.


In [44]:
# df['Title'].value_counts().to_dict() devuelve un diccionario con el titulo como key y la cantidad de ocurrencias como el valor
titulos = df['Title'].value_counts().to_dict()
# La funcion sorted es igual que en el ejercicio A.
print("Pelicula con mas ocurrencias: ")
print(dict(sorted(titulos.items(), key=lambda item: item[1], reverse=True)[:3]))
print()

# Por cada titulo unico sumamos la duracion de todas las ocurrencias
for titulo in titulos.keys():
  titulos[titulo] = sum(df[df['Title'] == titulo]['Duration'])

# La funcion sorted es igual que la anterior
print("Pelicula con mayor duracion total: ")
print(dict(sorted(titulos.items(), key=lambda item: item[1], reverse=True)[:3]))

Pelicula con mas ocurrencias: 
{'Community: Temporada 6: Consecuencias emocionales de las transmisiones televisivas (Capítulo 13)': 31, 'Community: Temporada 6: Espionaje moderno (Capítulo 11)': 31, 'Community: Temporada 6: Reparación de vehículos recreativos y quiromancia (Capítulo 10)': 26}

Pelicula con mayor duracion total: 
{'Star Wars: El Imperio contraataca': 29433, 'Iron Man: El Hombre de Hierro': 21718, 'Star Wars: Una nueva esperanza': 20394}


## C. Generar un TOP “N” de series vistas y graficar.
## 1.   Por tiempo visto
## 2.   Por ocurrencias

## D. Visualizar una semana de reproducciones genérica, en función a las reproducciones por día y por horario de todos los años.


## E. Contabilizar la cantidad de títulos vistos por año, graficar su variación a través de los años.


## F. Total de días de todos los títulos reproducidos.
